# Initialize

## Basic libraries

In [5]:
import os
from os.path import join

import numpy as np
import scipy as sp
#import pyqtgraph_extended as pg
import matplotlib.pyplot as plt
from matplotlib import pyplot
#import holography
import array as array
from scipy import signal

import sys
sys.path.append("C:\\_D\\python_codes\\analysis_code")
#import fthtools.masks as masks
#import fthtools.fth as fth
#import fthtools.PhaseRetrieval as PhR
import matplotlib.image as mpimg

# Self-written libraries
sys.path.append(join(os.getcwd(), "library"))
import mask_lib
import helper_functions as helper
import interactive
from interactive import cimshow

GPU available


## Custom functions

In [6]:
##Function to define stochastic noise
def gauss_2D(xx,yy,amp,sigma, x0,y0):
    return amp*np.exp(-((xx-x0)**2+(yy-y0)**2)/(sigma)**2)

##Function to convert rgb image to gray scale
def rgb2gray(rgb):
    return np.dot(rgb[...,:3], [0.2989, 0.5870, 0.1140])